In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import numpy as np
from sklearn.dummy import DummyClassifier
X = np.array([-1, 1, 1, 1])
y = np.array([0, 1, 1, 1])
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)

dummy_clf.predict(X)

dummy_clf.score(X, y)

test_data = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")
train_data = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
sample_data = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv")

from sklearn.dummy import DummyClassifier
X = train_data["ProductDescriptionPage_Duration"]
y = train_data["Made_Purchase"]
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(X)

sub = pd.DataFrame(dummy_clf.predict(test_data["ProductDescriptionPage_Duration"]), columns=['Made_Purchase'])
sub.index.name = 'id'
sub.to_csv("testOutput.csv", encoding='utf-8')

output = pd.read_csv("testOutput.csv")


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


In [29]:
train_data.dtypes

HomePage                           float64
HomePage_Duration                  float64
LandingPage                        float64
LandingPage_Duration               float64
ProductDescriptionPage             float64
ProductDescriptionPage_Duration    float64
GoogleMetric:Bounce Rates          float64
GoogleMetric:Exit Rates            float64
GoogleMetric:Page Values           float64
SeasonalPurchase                   float64
Month_SeasonalPurchase              object
OS                                 float64
SearchEngine                       float64
Zone                               float64
Type of Traffic                    float64
CustomerType                        object
Gender                              object
Cookies Setting                     object
Education                           object
Marital Status                      object
WeekendPurchase                    float64
Made_Purchase                         bool
dtype: object

In [30]:
train_data.shape

(14731, 22)

In [31]:
tt=train_data.replace('?', np.NaN)
tt.isnull().sum()

HomePage                           153
HomePage_Duration                  150
LandingPage                        153
LandingPage_Duration               135
ProductDescriptionPage             123
ProductDescriptionPage_Duration    167
GoogleMetric:Bounce Rates          151
GoogleMetric:Exit Rates            129
GoogleMetric:Page Values           132
SeasonalPurchase                   150
Month_SeasonalPurchase             144
OS                                 134
SearchEngine                       122
Zone                               117
Type of Traffic                    143
CustomerType                       144
Gender                             145
Cookies Setting                    144
Education                          136
Marital Status                     130
WeekendPurchase                    121
Made_Purchase                        0
dtype: int64

In [32]:
num_cols = ["HomePage","HomePage_Duration","LandingPage","LandingPage_Duration","ProductDescriptionPage","ProductDescriptionPage_Duration","GoogleMetric:Bounce Rates","GoogleMetric:Exit Rates","GoogleMetric:Page Values"]
cat_cols = ["OS","Month_SeasonalPurchase","SearchEngine","Zone","Type of Traffic","CustomerType", "Gender", 'Cookies Setting', 'Education', 'Marital Status', 'WeekendPurchase']

In [33]:
from sklearn.impute import SimpleImputer
train_data.replace('?', np.nan, inplace=True)
test_data.replace('?', np.nan, inplace=True)

In [34]:
#imputation
num_imputer = SimpleImputer(missing_values=np.nan,strategy='mean')
num_imputer = num_imputer.fit(train_data[num_cols])
train_data[num_cols]=num_imputer.transform(train_data[num_cols])
test_data[num_cols]=num_imputer.transform(test_data[num_cols])

cat_imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
cat_imputer = cat_imputer.fit(train_data[cat_cols])
train_data[cat_cols]=cat_imputer.transform(train_data[cat_cols])
test_data[cat_cols]=cat_imputer.transform(test_data[cat_cols])

train_data['SeasonalPurchase']=cat_imputer.fit_transform(np.array(train_data["SeasonalPurchase"]).reshape(-1,1))
test_data['SeasonalPurchase']=cat_imputer.transform(np.array(test_data["SeasonalPurchase"]).reshape(-1,1))

trd=train_data.isna().sum()
ted=test_data.isna().sum()

ted

HomePage                           0
HomePage_Duration                  0
LandingPage                        0
LandingPage_Duration               0
ProductDescriptionPage             0
ProductDescriptionPage_Duration    0
GoogleMetric:Bounce Rates          0
GoogleMetric:Exit Rates            0
GoogleMetric:Page Values           0
SeasonalPurchase                   0
Month_SeasonalPurchase             0
OS                                 0
SearchEngine                       0
Zone                               0
Type of Traffic                    0
CustomerType                       0
Gender                             0
Cookies Setting                    0
Education                          0
Marital Status                     0
WeekendPurchase                    0
dtype: int64

In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# create the OneHotEncoder instance
ohe = OneHotEncoder()
cat_data = ohe.fit_transform(train_data[cat_cols]).toarray()
cat_df = pd.DataFrame(cat_data, columns=ohe.get_feature_names_out(cat_cols))
# replace the original categorical columns with the new transformed data
train_data.drop(cat_cols, axis=1, inplace=True)
train_data = pd.concat([train_data, cat_df], axis=1)

train_data.shape

(14731, 89)

In [36]:
cattestdata = ohe.transform(test_data[cat_cols]).toarray()
cattestdf = pd.DataFrame(cattestdata, columns=ohe.get_feature_names_out(cat_cols))
# replace the original categorical columns with the new transformed data
test_data.drop(cat_cols, axis=1, inplace=True)
test_data = pd.concat([test_data, cattestdf], axis=1)

test_data.shape

(6599, 88)

In [37]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_data.drop("Made_Purchase",axis=1),train_data["Made_Purchase"],test_size=0.2,random_state=42)


In [38]:
from sklearn.preprocessing import StandardScaler

# create a StandardScaler object
scaler = StandardScaler()

# fit the scaler to the numerical features in training data
X_train[num_cols]=scaler.fit_transform(X_train[num_cols])

# transform the numerical features in training and test data
X_test[num_cols] = scaler.transform(X_test[num_cols])

test_data[num_cols]= scaler.transform(test_data[num_cols])


In [39]:
from sklearn.metrics import f1_score

In [40]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score,f1_score,classification_report

**Model 1**

In [41]:
# gbc=GradientBoostingClassifier(n_estimators=200, learning_rate=0.05, max_depth=5, min_samples_split=4, min_samples_leaf=2)

# gbc.fit(X_train,y_train)
# gbc.score(X_train,y_train)


In [42]:

# y_predgbc=gbc.predict(X_test)
# print(classification_report(y_test,y_predgbc))

**Model 2-Random Forest Classifier**

In [43]:
# rfc=RandomForestClassifier()
# rfc.fit(X_train,y_train)               
# rfc.score(X_train,y_train)



In [44]:
# y_predrfc=rfc.predict(X_test)
# print(classification_report(y_test,y_predrfc))

**Model 3-K Nearest Neighbour Classifier**

In [45]:
# knn=KNeighborsClassifier(n_neighbors=5)
# knn.fit(X_train,y_train)
# knn.score(X_train,y_train)



In [46]:
# y_predknn=knn.predict(X_test)
# print(classification_report(y_test,y_predknn))

**Model 4-Logistic Regression**

In [47]:
# lr=LogisticRegression(max_iter=1000)
# lr.fit(X_train,y_train)
# lr.score(X_train,y_train)

In [48]:
# y_predlr=lr.predict(X_test)
# print(classification_report(y_test,y_predlr))

**Model 5 perceptron**

In [49]:
# from sklearn.linear_model import Perceptron

# perceptron = Perceptron(tol=1e-3, random_state=42)
# perceptron.fit(X_train, y_train)
# perceptron.score(X_train, y_train)


In [50]:
# y_predppn = perceptron.predict(X_test)
# print(classification_report(y_test,y_predppn))

**model 6 gd**

In [51]:
# from sklearn.linear_model import SGDClassifier
# classifier = SGDClassifier(penalty='l2',loss='hinge')
# classifier.fit(X_train, y_train)
# classifier.score(X_train, y_train)

In [52]:
# ypred_clf = classifier.predict(X_test)
# print(classification_report(y_test,ypred_clf))

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier,StackingClassifier

# Define the base models
estimators = [('sgd', RandomForestClassifier(max_depth=500,n_estimators=1000)),
              ('gb', GradientBoostingClassifier(n_estimators=50,learning_rate=0.1)),
              ('lr',BaggingClassifier(LogisticRegression(solver='newton-cg')))]
                    

ensemble = StackingClassifier(estimators=estimators,final_estimator= SGDClassifier())
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)

print(classification_report(y_test, y_pred ))

              precision    recall  f1-score   support

       False       0.78      0.88      0.83      1885
        True       0.73      0.57      0.64      1062

    accuracy                           0.77      2947
   macro avg       0.75      0.72      0.73      2947
weighted avg       0.76      0.77      0.76      2947



In [54]:
submission5 = pd.DataFrame(ensemble.predict(test_data), columns=['Made_Purchase'])
submission5.index.name = 'id'
submission5.to_csv("Output.csv", encoding='utf-8')

output5 = pd.read_csv("Output.csv")

output5["Made_Purchase"].value_counts()

False    4600
True     1999
Name: Made_Purchase, dtype: int64